# Sistema de recomendaciones LicitaLAB
#### Basado en las descripciones de los usuarios y las categorías en que ofrecen.

Si es primera vez que se lanza este PyNotebook, se generarán un grupo de archivos, necesarios para llevar a cabo las recomendaciones. 

### Imports y configuraciones

In [6]:
import os
from utils.RecommenderSystem import RecommenderSystem  
import pandas as pd
from unidecode import unidecode 
from tqdm import tqdm

Agregar la ubicación donde se encuentra el .csv con el que se genera el espacio de usuarios.

In [10]:
CSV_PATH = 'C:\\Users\\magda\\OneDrive\\Escritorio\\MDS_licitalab\\query_final_results_20231026023937.csv'

Carga el Dataset.

In [11]:
print("Counting length of csv file")
with open(CSV_PATH, 'r', encoding='utf-8') as file:
    total_lines = sum(1 for _ in file)
print(f"{total_lines} rows will be loaded to create the recommendation system")
# Create an empty list to store the DataFrames from chunks
dfs = [] 
# Create a tqdm wrapper for pd.read_csv
with tqdm(total=total_lines, desc = 'Loading Dataset') as pbar:
    def update_progress(n):
        pbar.update(n)
    
    # Read the CSV file using pd.read_csv and provide the progress callback
    df_chunks = pd.read_csv(CSV_PATH, chunksize=100000, iterator=True, encoding='utf-8', nrows=total_lines)  # Specify the encoding
    for chunk in df_chunks:
        # Process each chunk if needed
        # You can access the chunk data in the 'chunk' DataFrame
        #chunk['first_two_digits_code'] = chunk['agilebuyingscode'].apply(lambda x: x[:2])
        chunk['feature_vector'] = chunk['agilebuyingscode'].apply(lambda x: x[:2]) + ' ' + chunk['agileoffereditemsdescripcionofertada']
        chunk['feature_vector'] = chunk['feature_vector'].apply(lambda x: unidecode(str(x)).lower())
        
        dfs.append(chunk)
        update_progress(chunk.shape[0])
df = pd.concat(dfs, ignore_index=True)

Counting length of csv file
34761793 rows will be loaded to create the recommendation system


Loading Dataset:  44%|████▍     | 15301056/34761793 [03:15<04:08, 78196.05it/s]


Lanzar el recomendador. Si no encuentra los archivos necesarios iniciará un proceso de generación de estos. Se demora bastante, pero no deberían haber problemas.

In [13]:
RS = RecommenderSystem(df,save_path=  os.getcwd())

Initializing Recommender System
The current directory is c:\Users\magda\OneDrive\Escritorio\MDS_licitalab\Modeling\hito_4
c:\Users\magda\OneDrive\Escritorio\MDS_licitalab\Modeling\hito_4
['count_vectorizer_model.pkl', 'dev_uservectorgen.ipynb', 'ExperimentalDash.py', 'kmeans_clusters.csv', 'kmeans_model.pkl', 'LaunchRS.py', 'ShowcaseSystem.ipynb', 'utils', 'vectorized_corpus.csv']
All necesary files have been found.


### Predecir recomendaciones

Se le entrega el string del rut al sistema recomendador a traves de la función `predict()`. El sistema retorna el número de cluster al que el usuario ha sido asignado y un dataframe con las compras ágiles recomendadas. 

In [14]:
n_cluster,gg = RS.predict('77.187.331-6') 


Unseen data point belongs to cluster 0


Agregando un poco de post-procesamiento se pueden encontrar visualizaciones de interés.

In [16]:
gg.describe()

,agilebuyingsavailableamount,agileitemsmp_id,agileoffereditemspreciototal,Cluster
count,1.530106e+07,1.530106e+07,1.530106e+07,679552.0
mean,9.890671e+05,4.649132e+07,2.357438e+05,0.0
std,6.195381e+05,4.077776e+06,3.391537e+05,0.0
min,1.000000e+00,4.212150e+07,0.000000e+00,0.0
25%,4.290000e+05,4.321151e+07,2.400000e+04,0.0
50%,9.000000e+05,4.412210e+07,8.880700e+04,0.0
75%,1.685220e+06,5.013170e+07,2.900000e+05,0.0
max,1.873500e+06,5.612200e+07,1.800000e+06,0.0
